In [3106]:
path ='/Users/taehunlee/Desktop/duty_schedule_ver2/2024_07_copy.xlsx'

In [3107]:
# from google.colab import drive
# drive.mount('/content/drive')

# Making new function or Updating old function for duty scheduler ver2

## Function to assign ICU duty before 1日休み

>1. Merit 

>> 1.1. member can take a longer holiday with afterday of ICU duty.

>2. Demerit

>> 2.1 Someone doesn't want ICU duty before ICU duty.

>3. Continuous Holiday Pattern

>>3.1 Sat, Sun, Mon as hoped holiday

>>3.2 Fri as hoped holiday, Sat, Sun

>>3.3 any weekday as hoped holiday and next day is public hoiday.

>>3.4 hoped holidays more than continuous 2 weekdays.

>4. Caution!!

>>4.1 If someone hopes a ICU day that is the 2 before day of the other member's hoped holiday, Priority is to the member hoping ICU duty at specific day.

>>> 4.1.2 And the member hoping holiday has addtional unavailable day at yesterday of hoped day.

>>4.2 

>5. Rule

>>5.1 a day before hoped holiday is automatically indicated as unavailable day if the member don't want any other type of duty.

>>5.2 If there is no ICU duty a day before 2 days of hoped holiday, the member is assigned as ICU duty.

>>5.3 But the member is a group of no_additional_duty or no_duty,  this rule is not applied to the member.

>>5.4 

# 1. 필요한 라이브러리 인스톨 및 로딩(Launching and Loading libraries)

In [3108]:
# !pip install holidays-jp
#There is API for JP Holidays to get information from google calendar. https://holidays-jp.github.io/
#If I USE Google calenader api, Maybe I will not need to update holidays of Japan
#https://pypi.org/project/japan_holiday/ It will be helpful to me
# !py -m pip list --oudated

OLD BOYS are ['北', '清水と', '荒井', '柴田', '井上', '上田']
No ICU members are ['北', '清水と', '荒井', '井上', '猪原']
No Cardiac members are ['北', '井上', '上田', '猪原', '小山']
Members of NO ADDITIONL duty are ['北', '井上', '上田', '猪原']


,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
old_boy,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
no_icu_duty,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
no_cardiac_surgery,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0
no_duty,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
no_additional_duty,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
weekend_icu,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1
weekend_call,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
num_icu_duty,0,0,0,3,0,0,1,3,3,0,3,3,3,3,3,3,3,3
num_call_duty,1,3,3,3,1,1,3,3,3,0,3,3,3,3,3,3,3,3
duty_num,1,3,3,3,1,4,4,4,4,4,4,4,4,4,4,4,4,4


In [ ]:
df_trans_hoped_day

In [4]:
#Function to make a dataframe having basic information of each member
#Dataframe is easider than dictionary. So, Ver2 will use dataframe if possible.

def make_members_info(df_mem_info, df_relation, df_trans_hoped_day):
    
    df_new_info = df_mem_info.copy()
    df_new_info = df_new_info.copy()
    no_duty_memb = df_new_info.columns[df_new_info.loc["no_duty"]== True]
    df_new_info.drop(no_duty_memb, axis=1, inplace=True)
    additional_rows = pd.DataFrame({col : None for col in df_new_info.columns},index= add_row_name)
    df_new_info = pd.concat([df_new_info, additional_rows])
    df_new_info.loc["duty_score"] = 10
    for name in df_new_info.columns:

        df_new_info.loc["no_relation", name] = list(df_relation.loc[df_relation[name] == True, name].index)

        df_new_info.loc["unavailable_days", name] = list(df_trans_hoped_day.loc[df_trans_hoped_day[name]==1,name].index.day)
        df_new_info.loc["hope_icu_duty", name] = list(df_trans_hoped_day.loc[df_trans_hoped_day[name]==2,name].index.day)
        df_new_info.loc["hope_call_duty", name] = list(df_trans_hoped_day.loc[df_trans_hoped_day[name]==3,name].index.day)

        if df_new_info.loc["no_icu_duty", name] ==1:
            df_new_info.loc["duty_score", name] -=5
    return df_new_info


IndentationError: expected an indented block (980402729.py, line 7)

In [5]:
result_dict

NameError: name 'result_dict' is not defined

# Test and Practice of function


## Function considering relation among members
> 1. Introduction
>> 1.1 There are some members having no good relation with someone.
>> 1.2. So, For good performance during night or holiday duty, Master should consider relationship between call duty and icu duty.
> 2. Architecture of this function
>> 2.1 Take a information from excel file that has relationship information of all members. 
>> 2.2 Insert additional key as no_relation to each name. AND make a list of name of members having not good relationship.
> 3. Caution!!!
>> 3.1 This information should be top secreted information. 
>>> 3.1.1 So, Master must keep this information with security.


In [3110]:
# # # #Making a dataframe from excel file with relation information 
# # df_relation = pd.read_excel(path, sheet_name='relation',header = 0)
# # df_relation = df_relation.dropna(axis = 0)
# # df_relation[df_relation["猪原"]== True]['name']

# #Insert list of name of colleagues with no good relationship to each member's key.
# #this code is in make_members_info()
# test_info = members_info.copy()
# for name in test_info.keys():
#     test_info[name]["no_relation"] = list(df_relation[df_relation[name]== True]['name'])

# df = pd.DataFrame(test_info)
# display(df)

# #Make a function to rule out pair of no good relationship
# def check_relation(name, day, members_info):
#     call_duty_name = name
#     icu_duty_name = None
#     for key, duty in members_info.items():
#         if day in duty['icu_duty']:
#             icu_duty_name = key
#     if icu_duty_name in members_info[call_duty_name]["no_relation"]:
#         print(f'Dueto personal circumstances, {icu_duty_name} of icu_duty of {day}  and  {call_duty_name} of call_duty cannot be paired .')
#         return False
#     else:
#         print(f'{icu_duty_name} of icu_duty of {day}  and  {call_duty_name} of call_duty is able to handle Cardiac surgery.')
#         return True

# 2. 여러번 반복사용하는 구문들은 함수로 정의(Functions for repeated code)

  -Get weekend of defined month
  -Get holiday of japan
  -remove_list(list1, list2)
  -check_list(list1, list2)
  -members_info_refresh(members,no_icu_duty,no_cardiac_surgery,unavavailable_days  )

## 2.1 Functions for making information table of members

In [205]:
#Function to make a dataframe having basic information of each member
#Dataframe is easider than dictionary. So, Ver2 will use dataframe if possible.

def make_members_info(df_mem_info, df_relation, df_trans_hoped_day):
    
    df_new_info = df_mem_info.copy()
    df_new_info = df_new_info.copy()
    no_duty_memb = df_new_info.columns[df_new_info.loc["no_duty"]== True]
    df_new_info.drop(no_duty_memb, axis=1, inplace=True)
    additional_rows = pd.DataFrame({col : None for col in df_new_info.columns},index= add_row_name)
    df_new_info = pd.concat([df_new_info, additional_rows])
    df_new_info.loc["duty_score"] = 10
    df_new_info.loc["no_holidays"] = 2
    for name in df_new_info.columns:

        df_new_info.loc["no_relation", name] = list(df_relation.loc[df_relation[name] == True, name].index)

        df_new_info.loc["unavailable_days", name] = list(df_trans_hoped_day.loc[df_trans_hoped_day[name]==1,name].index.day)
        df_new_info.loc["hope_icu_duty", name] = list(df_trans_hoped_day.loc[df_trans_hoped_day[name]==2,name].index.day)
        df_new_info.loc["hope_call_duty", name] = list(df_trans_hoped_day.loc[df_trans_hoped_day[name]==3,name].index.day)

        if df_new_info.loc["no_icu_duty", name] ==1:
            df_new_info.loc["duty_score", name] -=5
        if df_new_info.loc["old_boy", name] ==1:
            df_new_info.loc["no_holidays", name] -=1
    print(f"OLD BOYS are {list(df_new_info.columns[df_new_info.loc['old_boy'] ==True])}")
    print(f"No ICU members are {list(df_new_info.columns[df_new_info.loc['no_icu_duty'] ==True])}")
    print(f"No Cardiac members are {list(df_new_info.columns[df_new_info.loc['no_cardiac_surgery'] ==True])}")
    print(f"Members of NO ADDITIONL duty are {list(df_new_info.columns[df_new_info.loc['no_additional_duty'] ==True])}")
    return df_new_info


In [206]:
a = make_members_info(df_mem_info, df_relation, df_trans_hoped_day)

OLD BOYS are ['北', '清水と', '荒井', '柴田', '井上', '上田']
No ICU members are ['北', '清水と', '荒井', '井上', '猪原']
No Cardiac members are ['北', '井上', '上田', '猪原', '小山']
Members of NO ADDITIONL duty are ['北', '井上', '上田', '猪原']


In [207]:
no_duty_memb = df_mem_info.columns[df_mem_info.loc["no_duty"]== True]
no_duty_memb

Index(['小出', '松村', '清水ち', '竹山'], dtype='object')

Function Check if All of members can handle with holidays duty of this month ,Defining old boys have only once of holiday duty

>1.1 Equation of check availability of This month

>> 1.1.1 x: number of no no_duty members

>> 1.1.2 y: number of old boys

>> 1.1.3 z: total number of holiday duty of ICU and CALL

>> 1.1.4 Caution!

>>>1.1.4.1 There are specific members only having duty days assigned by themselves.

>>>>上田）only have a ICU duty assigned by himself and Call duty is only on weekday.

>>>>猪原）She has no additional duty except to days assigned by herself

>>>1.1.4.2 If there are enough availability, Old boy should be charged of 2 days of holiday duty. the method is defining order like (荒井＞清水と＞井上＞北) or assigning by Master(北)




>>1.1.5 So, Availability is same or more than 4. because there is possibility that 猪原 doesn't assign holiday duty by herself.

>>> $$(2x-y)-z > 3 $$

In [208]:
#Check available number of memebrs on holiday duty. And, if the members is deficient, Some members of old boys take a duty on holidays twice.
#This functions require 2 dataframes and 2 data of classified day and old boys.
#Compnay's availability for holiday duty is 32 times(duty member *2 + old boy *1) per month. So, Considering events that make some member unavailable of holiday duty, There are 4 times as a 

#Available total count of holiday duty days of members
    #1. Old boy has 1 count
    #2. No old boy has 2 count
        #2.1 No old boy except no_duty members has 2 count
        #2.2 So, total count is twice of members excluding old boys and no_duty members
    #3. But if Available total count is less than total count of holiday duty days, Old boy is charged of 2 counts.

def select_twice_old_boy(df_mem_info, 
                         info_days , 
                         members_df ,
                         old_boy):
    print(f"Old boys are {old_boy}")
    #count of no no_duty members as x
    total_memb = df_mem_info.columns[1:]
    total_num_memb = len(total_memb)

    #members able to do duty as x
    row_no_duty = df_mem_info.loc[df_mem_info["name"]=="no_duty"]
    num_df_memb_no_duty = row_no_duty.select_dtypes(include=["int"])
    no_duty_num = num_df_memb_no_duty.iloc[0].sum()
    total_able_number = total_num_memb - no_duty_num
    print(f"週末の当直ができる人員数：{total_able_number}")
    #members of old boy as y
    row_no_duty = df_mem_info.loc[df_mem_info["name"]=="old_boy"]
    num_df_memb_old_boy = row_no_duty.select_dtypes(include=["int"])
    old_boy_num = num_df_memb_old_boy.iloc[0].sum()
    print(f"Old boyで週末当直1回にする人員数：{old_boy_num}")

    #total count of duty days of holiday as z
    total_holi_duty = len(sorted(info_days[0] + info_days[2]+info_days[1]))*2
    print(f"今月に週末当直の総数：{total_holi_duty}")

    #Check eqauation : (2x-y)-z > 3
    holiday_availability = total_able_number*2-old_boy_num-total_holi_duty
    print(f"holiday_availability is {holiday_availability}")

    holiday_availability = 0
    old_boy_order = old_boy[::-1]
    required_num = 0
    # if holiday_availability<4:
    required_num = (4-holiday_availability)//2+(4-holiday_availability)%2
    print(f"Required number of twice old boy is {required_num}")
    twice_old_boy = old_boy_order[:required_num]
    print(f"Members of old boys take twice of duty on holiday {twice_old_boy}")
    for selected_memb in twice_old_boy:
        members_df.loc["no_holidays",selected_memb] = 0

    return members_df   

## 2.2 Funtions to Classify days and get a date information

In [209]:
#Extract day from excel string of date like 2/14(金)
def extract_day(date_str):
    # 正規表現で括弧とその中身を削除
    date_str = re.sub(r'\([^)]*\)', '', date_str).strip()
    # 日付をパースして日を抽出
    month, day = date_str.split('/')
    return day

#def additiional_holidays is removed becaus it is not used.

#Get a extra holiday from excel file(Checked)
    #Maybe this function is useful to input additional company's holiday without manipulating get_holidays_from_api

def get_extra_holidays(df_extra_info):
    extra_days = []
    dt = df_extra_info.loc[df_extra_info["extra_holidays"]==1].index
    for n in dt:
        print(n.day)
        extra_days.append(n.day)
    return extra_days


#Get a information of Japanese holiday through API based google calendar.
# Refrence : https://holidays-jp.github.io/ (MIT Licence)

def get_holidays_from_api(year):

    #Get a information about Japanese Holidays from Google calendar through by API
    api_url = f"https://holidays-jp.github.io/api/v1/{year}/date.json"
    response = requests.get(api_url)
    data = response.text
    parsed_data = json.loads(data)
    converted_data = {}
    for date_str, event in parsed_data.items():
        # Convert the date string to a datetime object
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        # Store the datetime object as the key in the new dictionary
        converted_data[date_obj] = event
    #Add holidays of Company
    lastyear_holidays = ((datetime(year,1,2,0,0), "年末年始休み"),
                        (datetime(year,1,3,0,0), "年末年始休み"),
                        (datetime(year,1,4,0,0), "年末年始休み"),
                        (datetime(year,12,30,0,0), "年末年始休み"),
                        (datetime(year,12,31,0,0), "年末年始休み"),)
    for holi_date, holi_name in lastyear_holidays:
        converted_data[holi_date] = holi_name

    sorted_converted_data = OrderedDict(sorted(converted_data.items(), key=lambda x: x[0]))
    print(f"Japanese Holidays of {year} : \n")
    return sorted_converted_data
#Holidays list from holidays-jp, Customized holidays of Company and Birthday of Jpanese King
# def holidays_of_this_year(year):

#     holi = CountryHolidays.get('JP', year)
#     if datetime(year,2,23,0,0).weekday() == 5:
#         holi.append((datetime(year,2,25,0,0),'振替休日'))
#     elif datetime(year,2,23,0,0).weekday() == 6:
#         holi.append((datetime(year,2,24,0,0),'振替休日'))
#     holi.append((datetime(year,2,23,0,0),'天皇誕生日'))

#     lastyear_holi = ((datetime(year,1,2,0,0), "年末年始休み"),
#                     (datetime(year,1,3,0,0), "年末年始休み"),
#                     (datetime(year,1,4,0,0), "年末年始休み"),
#                     (datetime(year,12,30,0,0), "年末年始休み"),
#                     (datetime(year,12,31,0,0), "年末年始休み"),)
#     for n in lastyear_holi:
#         holi.append(n)
#     sorted_holi = sorted(holi, key=lambda x: x[0])
#     return sorted_holi

# Check if first day of next month is holiday(Updated)

def check_first_day_of_next_month(year, month, holidays):
    next_month = month+1
    if month == 12:  #If this month is 12, make next_month 1
        next_month = 1
    holidays_date = list(holidays.keys())
    next_first = datetime(year, next_month, 1)
    if next_first in holidays_date:
        is_next_holiday = True
        print(f"the First day of Next Month is holiday")
        return is_next_holiday
    elif datetime(year,next_month,1,0,0).weekday() in [5,6]:
        is_next_holiday = True
        print(f"the First day of Next Month is holiday")
        return is_next_holiday
    else:
        is_next_holiday = False
        print(f"the First day of Next Month is NOT holiday")
        return is_next_holiday

#Divide days of selected month to weekdays and vacation_days including holidays
def distinct_days(year, month, extra_holiday, holidays):
    weekends = []
    day_holiday =[] #Removed useless variables#
    period = calendar.monthrange(year, month)[1]

    for day in range(1, period + 1):
        if calendar.weekday(year, month, day) in [5, 6]:  # 5=土曜日, 6=日曜日
            weekends.append(day)
    for date in holidays.keys():
            if date.month == month: #date is datetime type. So, date[0] is not needed
                day_holiday.append(date.day)
    else:
        # Assign a as a list containing None
        add_days = []
    if len(extra_holiday) != 0:
        print("There is extra holiday at this month.")
        integ_holiday = sorted(set(weekends + day_holiday + extra_holiday))
    if len(extra_holiday) == 0:
        print("There is NOT extra holiday at this month.")
        integ_holiday = sorted(set(weekends + day_holiday))
    print(f"Holidays of this month(休日) : {integ_holiday}")

    #Weekdays is remained days after Deleting holidays from of this month.
    weekdays = [day for day in range(1, period+1) if day not in integ_holiday]
    print(f"Weekdays of this month（平日） : {weekdays}")

    return weekdays, integ_holiday

#Classify weekday of a day before the holiday and the others(Updated)
def last_weekdays(weekdays, vacation_days, period, is_next_holiday):
    last_weekday = []
    no_last_weekdays = []
    for n in weekdays:
        if n+1 in vacation_days:
            last_weekday.append(n)
            print(f"{n} day is a last day of weekdays")

        elif weekdays[-1] == period\
            and is_next_holiday: #Remove unnecessary code
            last_weekday.append(n)
            print(f"{n} day is a last day of weekdays")

        else:
            print(f"{n} day is not a last day of weekdays")
            no_last_weekdays.append(n)

    return last_weekday, no_last_weekdays

#Determine if a day of vacation_days is last of long weekend.
def not_last_holiday(vacation_days, period, is_next_holiday):
    not_last_vacation_days = []
    last_vacation_day = []

    for n in vacation_days:
        if n+1 in vacation_days:
            not_last_vacation_days.append(n)
            print(f"{n} day is NOT a last day of vacation days")
            print(not_last_vacation_days)
        elif (n == vacation_days[-1] )\
            and vacation_days[-1] == period\
            and is_next_holiday:
            not_last_vacation_days.append(n)
            print(f"{n} day is NOT a last day of vacation_days")
            print(not_last_vacation_days)
        else:
            print(f"{n} day is a last day of vacation_days")
            last_vacation_day.append(n)

    return not_last_vacation_days, last_vacation_day

#Make a classified days by function and function(inform_days uses not_last_holiday and last_weekdays)
    #If the functions are sorted in order, a function can be nested within another function.

def inform_days(weekdays, vacation_days, period, is_next_holiday):
    not_last_vacation_days, last_vacation_days = not_last_holiday(vacation_days, period, is_next_holiday)
    last_weekday, not_last_weekdays = last_weekdays(weekdays, vacation_days, period, is_next_holiday)
    print('普通の平日 : ',not_last_weekdays )
    print('休日の前日の平日 : ', last_weekday)
    print('翌日平日である休日 : ', last_vacation_days)
    print('翌日が休日である休日 : ', not_last_vacation_days)

    return not_last_vacation_days, last_vacation_days, last_weekday, not_last_weekdays


## 2.3 Functions to Decide members randomly

In [269]:
def hoped_duty(members_df, info_days, icu = True):
    members_df = members_df.copy()
    icu_info_days = copy.deepcopy(info_days)
    call_info_days = copy.deepcopy(info_days)
    if icu :
        for name in members_df.columns:
            if members_df.loc['hope_icu_duty',name] :
                members_df.loc['icu_duty',name] = members_df.loc['hope_icu_duty',name]

                for day in members_df.loc['icu_duty',name]:
                    members_df.loc['duty_score',name] = count_down_duty_score(name, day, members_df,
                                                                              info_days, icu)
                for i in range(4):
                    icu_info_days[i] = remove_list(icu_info_days[i], members_df.loc["icu_duty",name])
    if icu == False:
        for name in members_df.keys():
            if members_df.loc['hope_call_duty', name] :
                members_df.loc['call_duty', name] = members_df.loc['hope_call_duty', name]

                for day in members_df.loc['hope_call_duty', name]:
                    members_df.loc['duty_score'] = count_down_duty_score(name, day, members_df,
                                                                              info_days, icu)
                for i in range(4):
                    call_info_days[i] = remove_list(call_info_days[i], members_df.loc["call_duty",name])
    return members_df, icu_info_days, call_info_days

In [211]:
#Decide duty members
def hoped_duty(members_df, info_days, icu = True):
    if icu :
        info_days[0] = info_days[0]
        info_days[1] = info_days[1]
        info_days[2] = info_days[2]
        info_days[3] = info_days[3]

        for name in members_df.keys():
            if members_df.loc['hope_icu_duty', name] :
                members_df.loc['icu_duty', name] = members_df.loc['hope_icu_duty', name]
                for day in members_df.loc['hope_icu_duty']:
                    members_df.loc['duty_score', name] = count_down_duty_score(name, day, members_df,
                                                                              info_days, icu)
                info_days[0] = remove_list(info_days[0], members_df.loc['icu_duty', name])
                info_days[1] = remove_list(info_days[1], members_df.loc['icu_duty', name])
                info_days[2] = remove_list(info_days[2], members_df.loc['icu_duty', name])
                info_days[3] = remove_list(info_days[3], members_df.loc['icu_duty', name])
    if icu == False:
        info_days[0] = info_days[0]
        info_days[1] = info_days[1]
        info_days[2] = info_days[2]
        info_days[3] = info_days[3]
        for name in members_df.keys():
            if members_df.loc['hope_call_duty', name] :
                for day in members_df.loc['hope_call_duty', name]:
                    members_df.loc['duty_score', name] = count_down_duty_score(name, day, members_df,
                                                                              info_days, icu)
                members_df.loc['call_duty', name] = members_df.loc['hope_call_duty']
                info_days[0] = remove_list(info_days[0], members_df.loc['call_duty', name])
                info_days[1] = remove_list(info_days[1], members_df.loc['call_duty', name])
                info_days[2] = remove_list(info_days[2], members_df.loc['call_duty', name])
                info_days[3] = remove_list(info_days[3], members_df.loc['call_duty', name])

def decide_duty_members(members_info,info_days, interval, icu=True):
    if icu:
        icu_info_days = info_days.copy()
        vacation_days = (info_days[0]+[1]).copy()
        left_info_days = info_days.copy()
        weekend = False
        try_num = 0
        for days in left_info_days:
            if try_num>50:
                print("try_num is {try_num}. Try Again, Master!!")
                break
            left_days = days.copy()
            print(f'{left_days}に対してICU当直を決めます。')
            if days in vacation_days:
                weekend == True
            choosen_day = None
            highest_members = []
            potential_day = None
            while left_days:
                while not highest_members:
                    max_score = highest_duty_score(members_info, icu)[1]
                    if max_score <-100:
                        print("try_num is {try_num}. Try Again, Master!!")
                        break
                    highest_members = [name for name in members_info.keys()
                                      if (members_info[f'{name}']['duty_score'] == max_score and
                                        members_info[f'{name}']['no_icu_duty'] == False
                                          )]
                try_num +=1
                print('try number: ',try_num)
                if try_num>50:
                    print("try_num is {try_num}. Try Again, Master!!")
                    break
                if try_num >10:
                    max_score -=1
                    add_mem = [name for name in members_info.keys()
                                    if members_info[name]['duty_score'] == max_score
                                    and members_info[name]['no_icu_duty'] == False]
                    print(f'{add_mem}を候補者に追加します。')
                    highest_members = highest_members + add_mem
                    try_num = 0
                if try_num >15:
                    max_score = highest_duty_score(members_info, icu)[1]
                    try_num = 0

                sorted_mem = sorted(highest_members,
                                    key=lambda x: (len(unavailabledays_of_member(x, members_info, weekend))),
                                    reverse=True)



                print(f'{sorted_mem}の中で{potential_day}のICU当直を決めます')
                if sorted_mem:
                    for potential_mem in sorted_mem:
                        if left_days:
                            potential_day = random.choice(left_days)
                        print(f'{potential_mem}と{potential_day}日のicu当直をチェックします.')
                        print(f' Max_score: {max_score}, 実施回数：{try_num}')
                        if check_not_unavailable_day(potential_day, potential_mem, members_info) \
                            and check_not_within_interval(potential_day, potential_mem, members_info,interval, icu)\
                            and check_no_holidays(potential_mem, members_info, potential_day, icu_info_days, icu)\
                            and check_duty_num(members_info, potential_mem):
                            choosen_mem = potential_mem
                            choosen_day = potential_day
                            print(f'remove {choosen_day} from {left_days}')
                            left_days.remove(choosen_day)
                            print(f'残りの当直日は{left_days}')

                            if choosen_day not in info_days[3]:
                                members_info[choosen_mem]['no_holidays'] +=1
                            members_info[choosen_mem]['duty_num'] -=1
                            members_info[choosen_mem]['icu_duty'].append(choosen_day)
                            members_info[choosen_mem]['duty_score'] = count_down_duty_score(choosen_mem, choosen_day, members_info, left_info_days, icu)
                            highest_members = [name for name in members_info.keys()
                                                if members_info[name]['duty_score'] == max_score
                                                and members_info[name]['no_icu_duty'] == False]
                            print(f'{choosen_mem}は{choosen_day}日のICU当直ができます')
                            print('>>>>>>>>>'*10)
                            if not left_days:
                                break
                        else:
                            print(f'{potential_mem}は{potential_day}日のicu当直ができません.')
                            pass


            print('*************'*20)
            print('*************'*20)
        print('#$%&'*100)
    if icu == False:
        call_info_days = info_days.copy()
        vacation_days = (info_days[0]+[1]).copy()
        left_info_days = info_days.copy()
        weekend = False
        try_num = 0
        for days in left_info_days:
            if try_num>50:
                print("try_num is {try_num}. Try Again, Master!!")
                break
            left_days = days.copy()
            print(f'{left_days}に対してCALL当直を決めます。')
            if days in vacation_days:
                weekend == True
            choosen_day = None
            highest_members = []
            potential_day = None
            while left_days:
                if try_num>50 :
                    print("try_num is {try_num}. Try Again, Master!!")
                    break
                while not highest_members:
                    max_score = highest_duty_score(members_info, icu)[1]
                    if max_score <-100:
                        print("try_num is {try_num}. Try Again, Master!!")
                        break
                    highest_members = [name for name in members_info.keys()
                                      if (members_info[f'{name}']['duty_score'] == max_score
                                          )]
                try_num +=1
                print('try number: ',try_num)
                if try_num >10:
                    max_score -=1
                    add_mem = [name for name in members_info.keys()
                                    if members_info[name]['duty_score'] == max_score]
                    print(f'{add_mem}を候補者に追加します。')
                    highest_members = highest_members + add_mem
                    try_num = 0
                if try_num >15:
                    max_score = highest_duty_score(members_info, icu)[1]
                    try_num = 0


                sorted_mem = sorted(highest_members,
                                    key=lambda x: (members_info[x]['no_icu_duty'],
                                                   len(unavailabledays_of_member(x, members_info, weekend))),
                                    reverse=True)

                print(f'{sorted_mem}の中で{potential_day}のCall当直を決めます')
                if sorted_mem:
                    for potential_mem in sorted_mem:
                        if left_days:
                            potential_day = random.choice(left_days)
                        print(f'{potential_mem}と{potential_day}日のCall当直をチェックします.')
                        print(f' Max_score: {max_score}, 実施回数：{try_num}')
                        if check_not_unavailable_day(potential_day, potential_mem, members_info) \
                            and check_not_within_interval(potential_day, potential_mem, members_info,interval, icu)\
                            and check_cardiac_pair(potential_mem, potential_day, members_info)\
                            and check_old_boy_pair(potential_mem, potential_day, members_info)\
                            and check_no_holidays(potential_mem, members_info, potential_day,call_info_days, icu)\
                            and check_relation(potential_mem, potential_day, members_info)\
                            and check_duty_num(members_info, potential_mem):

                            choosen_mem = potential_mem
                            choosen_day = potential_day
                            print(f'remove {choosen_day} from {left_days}')
                            left_days.remove(choosen_day)
                            print(f'残りの当直日は{left_days}')
                            if choosen_day not in info_days[3]:
                                 members_info[choosen_mem]['no_holidays'] +=1
                            members_info[choosen_mem]['duty_num'] -=1
                            members_info[choosen_mem]['call_duty'].append(choosen_day)
                            members_info[choosen_mem]['duty_score'] = count_down_duty_score(choosen_mem, choosen_day,
                                                                                            members_info, left_info_days, icu)
                            highest_members = [name for name in members_info.keys()
                                                if members_info[name]['duty_score'] == max_score]
                            print(f'{choosen_mem}は{choosen_day}日のcall当直ができます')
                            print('>>>>>>>>>'*10)
                            if not left_days:
                                break
                        else:
                            print(f'{potential_mem}は{potential_day}日のcall当直ができません.')
                            pass


            print('*************'*20)
            print('*************'*20)
        print('#$%&'*100)




## 2.4 Function to check the rules for deciding duty members

In [212]:
#count down duty_score by duty class and duty day
def count_down_duty_score(name, day, members_df, info_days, icu= True):
    criteria_days = info_days.copy()
    score= 0
    duty_score = members_df.loc['duty_score', name]
    if icu:
        if day in criteria_days[0]:
            score = 10
        elif day in criteria_days[1]:
            score = 8
        elif day in criteria_days[2]:
            score = 9
        elif day in criteria_days[3]:
            score = 4
    else:
        if day in criteria_days[0]:
            score = 6
        elif day in criteria_days[1]:
            score = 6
        elif day in criteria_days[2]:
            score = 3
        elif day in criteria_days[3]:
            score = 3
    duty_score -= score
    print(f'day : {day}, score = {score}')
    print(f"{name}'s duty_score(after) : {duty_score +score} -> {duty_score}")
    return duty_score

#Check what is a highest duty_score of all members,
def highest_duty_score(members_info, icu = True):
    if icu:
        max_score = max(name['duty_score'] for name in members_info.values() if name['no_icu_duty'] != icu)
        keys = [key for key, value in members_info.items() if value['duty_score'] == max_score and value['no_icu_duty'] != icu]
    else:
        max_score = max(name['duty_score'] for name in members_info.values())
        keys = [key for key, value in members_info.items() if value['duty_score'] == max_score]
    return keys, int(max_score)


#Check the duty day if the day is contraindicated

#count the number of unavailable day on weekday or weekend
def unavailabledays_of_member(name, members_info, weekend = True):
    if weekend == True:
        left_days = remove_list(members_info[f'{name}']['unavailable_days'], weekdays)
    elif weekend ==False:
        left_days = remove_list(members_info[f'{name}']['unavailable_days'], vacation_days)

    return left_days

##Modulse for each contraindication

#Check if the day is in unavailable_day
def check_not_unavailable_day(choosen_day, choosen_member, members_info):
    print(choosen_day, choosen_member)
    if check_list(members_info[choosen_member]['unavailable_days'], [choosen_day]):
        print(f'{choosen_day} is unavailable day of {choosen_member}')
        return False
    else:
        return True


#Check if the day is within 3days after a previous duty day.
def check_not_within_interval(choosen_day, choosen_member, members_info, interval, icu = True):
    period = list(range(choosen_day-interval, choosen_day+interval+1))
    print(f'Days is {period}')
    if icu == True:
        print(f"{choosen_member}'s ICU duty is {members_info[choosen_member]['icu_duty']}")
        if check_list(members_info[choosen_member]['icu_duty'], period) :
            print(f"the {choosen_day} is contraindicated because within {interval} days from previous icu duty day")
            return False
        elif check_list(members_info[choosen_member]['call_duty'], period) :
            print(f"the {choosen_day} is contraindicated because within {interval} days from previous call duty day")
            return False
        else:
            return True
    if icu == False:
        print(f"{choosen_member}'s CALL duty is {members_info[choosen_member]['icu_duty']}")
        if check_list(members_info[choosen_member]['call_duty'], period) :
            print(f"the {choosen_day} is contraindicated because within {interval} days from previous call duty day")
            return False
        elif check_list(members_info[choosen_member]['icu_duty'], period) :
            print(f"the {choosen_day} is contraindicated because within {interval} days from previous icu duty day")
            return False
        else:
            return True
# def check_not_within_two(choosen_day, choosen_member, members_info, icu = True):
#     if icu == True:
#         if check_list(members_info[f'{choosen_member}']['icu_duty'], [choosen_day-3,choosen_day-2, choosen_day-1,choosen_day,choosen_day+1,choosen_day+2,choosen_day+3]) :
#             print(f'the {choosen_day} is contraindicated because within 2days from previous icu duty day')
#             return False
#         elif check_list(members_info[f'{choosen_member}']['call_duty'], [choosen_day-3,choosen_day-2, choosen_day-1,choosen_day,choosen_day+1,choosen_day+2]) :
#             print(f'the {choosen_day} is contraindicated because within 2days from previous call duty day')
#             return False
#         else:
#             return True
#     if icu == False:
#         if check_list(members_info[f'{choosen_member}']['call_duty'], [choosen_day-3,choosen_day-2, choosen_day-1,choosen_day,choosen_day+1,choosen_day+2]) :
#             print(f'the {choosen_day} is contraindicated because another call duty day is within 2days')
#             return False
#         elif check_list(members_info[f'{choosen_member}']['icu_duty'], [choosen_day-3,choosen_day-2, choosen_day-1,choosen_day,choosen_day+1,choosen_day+2]) :
#             print(f'the {choosen_day} is contraindicated because another icu duty day is within 2days')
#             return False
#         else:
#             return True


#Check if the confirmed duty team can handle cardiac surgery
def check_cardiac_pair(name, day, members_info):
    call_duty_name = name
    icu_duty_name = None
    for key, duty in members_info.items():
        if day in duty['icu_duty']:
            icu_duty_name = key
    if members_info[call_duty_name]['no_cardiac_surgery'] and members_info[icu_duty_name]['no_cardiac_surgery']:
        print(f'{icu_duty_name} of icu_duty of {day}  and  {call_duty_name} of call_duty is not able to handle Cardiac surgery.')
        return False
    else:
        print(f'{icu_duty_name} of icu_duty of {day}  and  {call_duty_name} of call_duty is able to handle Cardiac surgery.')
        return True

#Check if the number of no_holidays is over 2.
def check_no_holidays(name, members_info, potential_day, info_days, icu):
    if icu:
        if members_info[name]['no_holidays'] == 2 and potential_day not in info_days[3]:
            print("週末が2回以上えられたため当直の禁忌です")
            return False
        else:
            return True
    if icu == False:
        if members_info[name]['no_holidays'] == 2 and potential_day not in  info_days[3]:
            print("週末が2回以上えられたため当直の禁忌です")
            return False
        else:
            return True

def check_no_additional_duty(name, members_info):
    if members_info[name]["no_additional_duty"] ==1:
        return False
    else:
        return True

#Check if number of duty is over limitation of each members.
def check_duty_num(membs_info, name):
    if membs_info[name]["duty_num"] >=1:
        return True
    else:
        print(f"Total number of duty is over 5 days. So {name} can't be assigned to another duty anymore")
        return False

#Check if both members of pair are old boy
def check_old_boy_pair(name, day, members_info):
    call_duty_name = name
    icu_duty_name = None
    for key, duty in members_info.items():
        if day in duty['icu_duty']:
            icu_duty_name = key
    if members_info[call_duty_name]['old_boy'] and members_info[icu_duty_name]['old_boy']:
        print(f'Both {icu_duty_name} and {call_duty_name} at {day} day  is OLD BOY.')
        return False
    else:
        print(f'Either {icu_duty_name} or {call_duty_name} at {day} is not OLD BOY.')
        return True
    
    #Make a function to rule out pair of no good relationship
def check_relation(name, day, members_info):
    call_duty_name = name
    icu_duty_name = None
    for key, duty in members_info.items():
        if day in duty['icu_duty']:
            icu_duty_name = key
    if icu_duty_name in members_info[call_duty_name]["no_relation"]:
        print(f'Due to personal circumstances, {icu_duty_name} of icu_duty of {day}  and  {call_duty_name} of call_duty cannot be paired .')
        return False
    else:
        print(f'{icu_duty_name} of icu_duty of {day}  and  {call_duty_name} of call_duty have good relationship each other.')
        return True

## 2.4 Functions to make a form of dataframe, date table, calendar and excel file

In [213]:
#Make a dataframe of duty days of all members
def make_schedule_df(members_info):
    confirmed_schedule = pd.DataFrame(index=range(1,period+1), columns = ['icu_duty','call_duty'])
    for name in members_info.keys():
        for n in members_info[f'{name}']['icu_duty']:
            confirmed_schedule['icu_duty'][n] = name
    for name in members_info.keys():
        for n in members_info[f'{name}']['call_duty']:
            confirmed_schedule['call_duty'][n] = name
    print(confirmed_schedule)
    return confirmed_schedule

#Make a dataframe as a shape of calendar
def make_calendar(table, year, month, start, end):
    df = table
    # Create a date range for January 2024
    dates = pd.date_range(start=start, end=end)

    # Assign the dates to the dataframe
    df['date'] = dates

    # Create a calendar
    cal = calendar.monthcalendar(year, month)

    # Create a new dataframe for the calendar
    calendar_df = pd.DataFrame(index=pd.MultiIndex.from_product([['1st week', '2nd week', '3rd week', '4th week', '5th week','6th week'], ['day','icu_duty', 'call_duty']], names=['Week', 'Duty']), columns=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

    # Fill the calendar dataframe with the duties
    for i, week in enumerate(cal):
        for j, day in enumerate(week):
            if day != 0:  # Calendar's month days are represented as 0
                duty = df[df['date'].dt.day == day]
                if day in vacation_days:
                    calendar_df.loc[(f'{i+1}st week', 'day'), calendar_df.columns[j]] = f'*{day}'
                else:
                    calendar_df.loc[(f'{i+1}st week', 'day'), calendar_df.columns[j]] = day
                calendar_df.loc[(f'{i+1}st week', 'icu_duty'), calendar_df.columns[j]] = duty['icu_duty'].values[0]
                calendar_df.loc[(f'{i+1}st week', 'call_duty'), calendar_df.columns[j]] = duty['call_duty'].values[0]

    # Drop the weeks where all duties are NaN
    calendar_df.dropna(how='all', inplace=True)

    # Print the calendar
    print(calendar_df)
    return calendar_df

## 2.5 Functions to simplify function

In [214]:
#remove elements in list2 from list 1
def remove_list(list1, list2):
    set2 = set(list2)
    return [element for element in list1 if element not in set2]

#check if there is element of list2 in list1
def check_list(list1, list2):
    intersection = [n for n in list1 if n in list2]
    print(intersection)
    if len(intersection) == 0:
        return False
    else:
        return True

#Return elements shared between list1 and list2

def shared_elements(list1, list2):
    common_elements = set(list1).intersection(set(list2))
    return list(common_elements)


## 2.6 Deprecated Functions

In [215]:


# def make_members_info(df_trans_hoped_day, df_mem_info, df_relation):
#     members_info = {}
#     for name in df_mem_info.columns[1:]:

#     #Take a information about skill or limitation of all members from dataframe
    
#     no_cardiac_surgery = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_cardiac_surgery'] == 1).iloc[0]])
#     no_icu_duty = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_icu_duty'] == 1).iloc[0]])
#     no_additional_duty = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_additional_duty'] == 1).iloc[0]])
#     old_boy = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'old_boy'] == 1).iloc[0]])
#     #Check if information of members about skill and limitation is correct
#     print("心臓手術非対応 : ",no_cardiac_surgery)
#     print("ICU当直非対応 : ",no_icu_duty)
#     print("指定Duty以外は追加無し : ",no_additional_duty)
#     print("OLD BOY : ",old_boy)

#     #Make a dict item. Key is a name of each member and Values are any information of skill, limitation, hoped_duty and duty_day
#     for name in df_trans_hoped_day.columns:

#         #make a base frame of dict
#         members_info[name] = {
#             'no_icu_duty' : False,
#             'no_cardiac_surgery' :False,
#             'no_additional_duty' : False,
#             "old_boy" : False,
#             'unavailable_days': [],
#             'hope_icu_duty': [],
#             'hope_call_duty': [],
#             'icu_duty' :[],
#             'call_duty' :[],
#             'duty_score' :11,
#             'no_holidays' :0,
#             'duty_num': 0,
#             }
#         #Insert list of name of colleagues with no good relationship to each member's key.
#         #this code is in make_members_info()
#         members_info[name]["no_relation"] = list(df_relation[df_relation[name]== True]['name'])

#         #Add information about duty days. the data is datetime type
#         members_info[name]['unavailable_days'].extend(df_trans_hoped_day[df_trans_hoped_day[name] == 1].index.tolist())
#         members_info[name]['hope_icu_duty'].extend(df_trans_hoped_day[df_trans_hoped_day[name] == 2].index.tolist())
#         members_info[name]['hope_call_duty'].extend(df_trans_hoped_day[df_trans_hoped_day[name] == 3].index.tolist())

#         #Insert a duty_num inputed for each member by Master
#         members_info[name]['duty_num']= int(df_mem_info.loc[df_mem_info['name']=='duty_num',name])

#         #Extract only the day from datetime to use only number.
#         members_info[name]['unavailable_days']= [n.day for n in members_info[name]['unavailable_days']]
#         members_info[name]['hope_icu_duty']= [n.day for n in members_info[name]['hope_icu_duty']]
#         members_info[name]['hope_call_duty']= [n.day for n in members_info[name]['hope_call_duty']]

#         #Adjust basic information based on characteristics of Each Member
#         if name in no_icu_duty: #Set duty_ score of the members in no_icu_duty list. unless set the score like this the members get so many call duty.
#             members_info[name]['no_icu_duty'] = True
#             members_info[name]['duty_score'] = 5 # This point is set under of base score.
#         if name in no_cardiac_surgery: #
#             members_info[name]['no_cardiac_surgery'] = True
#         if name in no_additional_duty: #Set the duty_score of members in no_additional_duty list to exclude additional selection from the duty schedule
#             members_info[name]['no_additional_duty'] = True
#             members_info[name]['duty_score'] = -100
#         if name in old_boy: #Set the no_holidays as 1 because in this algorithm old boy will get 2 duty on holidays.
#             members_info[name]['old_boy'] = True
#             members_info[name]['no_holidays'] = 1 #But if there are many days of holidays, old boy also get a duty work on 2 holidays.

#     return members_info

In [216]:
# #Take information from dataframe containing characteristics of each member
# def list_memb_skill(df_mem_info):
#     no_cardiac_surgery = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_cardiac_surgery'] == 1).iloc[0]])
#     no_icu_duty = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_icu_duty'] == 1).iloc[0]])
#     no_additional_duty = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_additional_duty'] == 1).iloc[0]])
#     old_boy = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'old_boy'] == 1).iloc[0]])
#     return no_cardiac_surgery, no_icu_duty, no_additional_duty , old_boy

#  3. 만들고자 하는 년월의 정보 로딩

## 3.1 Make a dataframe from excel file And Take a information

In [3119]:
#Take a information of Year and Month that user wants to use
df_date = pd.read_excel(path, sheet_name = 'hoped_day', nrows = 1, header = None)
year = df_date.iloc[0,1]
month = df_date.iloc[0,3]
_, month_range = calendar.monthrange(year, month)
print(f"This schedule is of {year}/{month}")

This schedule is of 2024/7


In [3120]:
#Load Excel file and take a information of date.
df_hoped_day= pd.read_excel(path, sheet_name='hoped_day',header = 1,nrows=month_range, index_col=0)
df_hoped_day.drop(0,axis = 1, inplace=True)
start_index = df_hoped_day.index[df_hoped_day['日付'].isna()].min()
start_column = df_hoped_day.columns[2]
end_column = df_hoped_day.columns[-1]
num_memb = len(df_hoped_day.columns)-1
name_of_memb = list(df_hoped_day.columns[1:])

columns_range = df_hoped_day.loc[:, start_column:end_column]
# Survey unique data within the specified range of columns
unique_values = columns_range.stack().unique()

df_hoped_day

,日付,祝日,北,清水と,荒井,柴田,井上,上田,一澤,西村,...,小出,竹田,松村,寺西,清水ち,髙田,池田,久保,李,竹山
0,2024-07-01,NaN,NaN,NaN,NaN,NaN,直開け,ICU,NaN,直開け,...,NaN,NaN,PM休,直開け,NaN,NaN,X,NaN,直開け,X
1,2024-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ICU,...,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,直開け,NaN
2,2024-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,１日休,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-07-05,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,１日休,...,NaN,NaN,NaN,本院,NaN,X,NaN,X,NaN,NaN
5,2024-07-06,麻酔科学会,NaN,出張,X,NaN,CALL,NaN,NaN,X,...,NaN,X,NaN,NaN,NaN,X,NaN,X,NaN,NaN
6,2024-07-07,麻酔科学会,NaN,出張,X,NaN,NaN,NaN,X,X,...,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN
7,2024-07-08,麻酔科学会,１日休,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN
8,2024-07-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-07-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3121]:
select_class = {1 : ['外勤', 'X', '１日休', '1日休み', '~16', '~17', '~13', '学会',"出張",'~15', '本院',"夏休","直開け"],
                2 : ['ICU',"休み前"],
                3 : ['CALL']}

#Value and keys can be exchanged by under code.
value_to_key = {val : key for key, values in select_class.items() for val in values}
#By dictionary, transfor a Key to value and make type int. And if there is no key in dict, np.nan will be 0.
df_trans_hoped_day = df_hoped_day.loc[:, start_column:end_column].applymap(lambda x : int(value_to_key.get(x,0)))
df_trans_hoped_day.index= df_hoped_day["日付"]
df_trans_hoped_day

/var/folders/h5/jyq7r4dd38l9x866mmggyx280000gn/T/ipykernel_1010/2722849129.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_trans_hoped_day = df_hoped_day.loc[:, start_column:end_column].applymap(lambda x : int(value_to_key.get(x,0)))


,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,...,小出,竹田,松村,寺西,清水ち,髙田,池田,久保,李,竹山
日付,,,,,,,,,,,,,,,,,,,,,
2024-07-01,0,0,0,0,1,2,0,1,0,1,...,0,0,0,1,0,0,1,0,1,1
2024-07-02,0,0,0,0,0,0,0,2,0,1,...,0,0,0,1,0,0,0,0,1,0
2024-07-03,0,0,0,0,0,0,1,0,1,2,...,0,0,0,0,0,0,0,0,0,0
2024-07-04,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-07-05,0,0,1,0,0,0,0,1,1,1,...,0,0,0,1,0,1,0,1,0,0
2024-07-06,0,1,1,0,3,0,0,1,1,1,...,0,1,0,0,0,1,0,1,0,0
2024-07-07,0,1,1,0,0,0,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0
2024-07-08,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2024-07-09,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [172]:
#Load excel file and take a information of extra_info
df_extra_info= pd.read_excel(path, sheet_name='extra_info',header = 0,index_col = 0)
df_extra_info

,extra_holidays,no_second_hospital
date,,
2024-07-01,0.0,0.0
2024-07-02,0.0,0.0
2024-07-03,0.0,0.0
2024-07-04,0.0,0.0
2024-07-05,0.0,0.0
2024-07-06,0.0,0.0
2024-07-07,0.0,0.0
2024-07-08,0.0,0.0
2024-07-09,0.0,0.0


In [107]:
#Take a information of mem_info(outside work, availability of duty,icu_duty and cardiac surgery) from excel file
df_mem_info = pd.read_excel(path, sheet_name='mem_info',header = 0,index_col=0)
# df_mem_info.loc["no_relation", ]= 
df_mem_info

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,...,小出,竹田,松村,寺西,清水ち,髙田,池田,久保,李,竹山
name,,,,,,,,,,,,,,,,,,,,,
old_boy,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
no_icu_duty,1,1,1,0,1,0,0,0,0,1,...,1,0,1,0,1,0,0,0,0,1
no_cardiac_surgery,1,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
no_duty,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
no_additional_duty,1,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
weekend_icu,0,0,0,1,0,0,1,1,1,1,...,0,1,0,1,0,1,1,1,1,0
weekend_call,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,1,1,1,1,0
num_icu_duty,0,0,0,3,0,0,1,3,3,0,...,0,3,0,3,0,3,3,3,3,0
num_call_duty,1,3,3,3,1,1,3,3,3,0,...,0,3,0,3,0,3,3,3,3,0


In [3124]:
#Delete members of no_duty from df_trans_hoped_day
no_duty = list(df_mem_info.columns[(df_mem_info[df_mem_info['name'] == 'no_duty'] == 1).iloc[0]])
df_trans_hoped_day = df_trans_hoped_day.drop(columns = no_duty)
df_trans_hoped_day

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
日付,,,,,,,,,,,,,,,,,,
2024-07-01,0,0,0,0,1,2,0,1,0,1,0,1,0,1,0,1,0,1
2024-07-02,0,0,0,0,0,0,0,2,0,1,0,1,0,1,0,0,0,1
2024-07-03,0,0,0,0,0,0,1,0,1,2,0,0,0,0,0,0,0,0
2024-07-04,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2024-07-05,0,0,1,0,0,0,0,1,1,1,1,0,0,1,1,0,1,0
2024-07-06,0,1,1,0,3,0,0,1,1,1,1,0,1,0,1,0,1,0
2024-07-07,0,1,1,0,0,0,1,1,0,1,1,0,0,0,1,0,0,0
2024-07-08,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0
2024-07-09,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0


In [176]:
# #Making a dataframe from excel file with relation information 
df_relation = pd.read_excel(path, sheet_name='relation',header = 0, index_col=0)
df_relation = df_relation.dropna(axis = 0)
df_relation

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,...,小出,竹田,松村,寺西,清水ち,髙田,池田,久保,李,竹山
name,,,,,,,,,,,,,,,,,,,,,
北,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
清水と,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
荒井,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
柴田,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
井上,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
上田,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
一澤,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
西村,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
孫,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [177]:
#Take a information of outpatient duty doctor from excel file
df_outpatient = pd.read_excel(path, sheet_name='outpatient',header = 0, index_col=0)
df_outpatient

/Users/taehunlee/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,mon,tue,wed,thu,fri
name,,,,,
pain_outpatient,一澤,池田,松村,井上,井上
preop_outpatient,北,猪原,寺西,竹田,一澤


In [178]:
#Take a information of available weekdays of outside work
df_outside_work = pd.read_excel(path, sheet_name='outside_work',header = 0, index_col= 0)
df_outside_work

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,...,小出,竹田,松村,寺西,清水ち,髙田,池田,久保,李,竹山
name,,,,,,,,,,,,,,,,,,,,,
中津,0,1,0,0,0,0,0,0,1,0,...,0,1,1,0,1,0,1,1,1,1
千船,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
南大阪,0,0,0,0,0,0,1,1,0,0,...,0,1,1,0,1,0,0,0,0,0
貝塚,0,0,1,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
伊丹,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
外勤回数,2,2,2,2,2,0,2,3,3,3,...,3,0,3,0,3,0,3,3,3,3
第２回数,0,0,0,0,0,1,1,1,1,1,...,1,1,1,0,1,0,1,1,1,1


--------------------------------------------------------------------------------------------------------------------------------------------

## 3.2 Make a applied information based on loaded data

In [255]:
#Get a information from excel file by using function
holidays = get_holidays_from_api(year)
is_next_holiday = check_first_day_of_next_month(year, month, holidays)
extra_holiday = get_extra_holidays(df_extra_info)
#Define year and month. And monthrange can be achievied by calendar library.
period = calendar.monthrange(year, month)[1]
start = datetime(year, month, 1).strftime('%Y-%m-%d')
end = datetime(year,month, period).strftime('%Y-%m-%d')

Japanese Holidays of 2024 : 

the First day of Next Month is NOT holiday


## 3.3 Classify days based on calendar and extra holidays

In [256]:
#Classify days to weekdays and holidays
weekdays, vacation_days = distinct_days(year, month, extra_holiday, holidays)

#Classify days by criteria

#List of lists of classified days. Order is weight decreased
info_days = list(inform_days(weekdays, vacation_days, period, is_next_holiday))
# preday, weekdays = get_preday_of_holiday(weekdays, vacation_days)

There is NOT extra holiday at this month.
Holidays of this month(休日) : [6, 7, 13, 14, 15, 20, 21, 27, 28]
Weekdays of this month（平日） : [1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 16, 17, 18, 19, 22, 23, 24, 25, 26, 29, 30, 31]
6 day is NOT a last day of vacation days
[6]
7 day is a last day of vacation_days
13 day is NOT a last day of vacation days
[6, 13]
14 day is NOT a last day of vacation days
[6, 13, 14]
15 day is a last day of vacation_days
20 day is NOT a last day of vacation days
[6, 13, 14, 20]
21 day is a last day of vacation_days
27 day is NOT a last day of vacation days
[6, 13, 14, 20, 27]
28 day is a last day of vacation_days
1 day is not a last day of weekdays
2 day is not a last day of weekdays
3 day is not a last day of weekdays
4 day is not a last day of weekdays
5 day is a last day of weekdays
8 day is not a last day of weekdays
9 day is not a last day of weekdays
10 day is not a last day of weekdays
11 day is not a last day of weekdays
12 day is a last day of weekdays
16 day i

# 4. 각 회사의 직원들의 기본 정보와 그눔 희망날짜, 피하고 싶은 날짜, 가능한 능력에 대해 직접 입력

## 4.1 기본 정보 입력 및 입력 된 정보를 DataFrame으로 확인

# 5. 당직일자를 정하기

> 1) 희망 당직날짜부터 배정

> 2) 주말 및 휴일 당직 배정

> 3) duty_score, 희망 당직일, 피하고 싶은 당직일 등 문제가 없는지 최종확인


## 5.0 당직의 규칙 정하기(new_version)

>1) icu당직을 한 경우, 다음날을 포함 2일간은 icu 및 call당직을 하지 않는다.

>2)call당직을 한 경우, icu당직은 다음날에 하지 않으며, 다음 call 당직은 최소 2일간은 하지 않음.

>3)call 당직자와 icu 당직자 중 한명은 심장수술 대응이 가능해야 함.

>3) 희망일을 우선 배정하지만, 겹치지 않는 다는 전제에 함. 그렇기 때문에 당직날을 정하기 전에 당사자간 협의하도록 한다.

>4) 휴일 icu당직은 기본 1일로 정하되, 사람이 부족한 경우에는 2일을 할 수 있음.

>) 평일 icu당직은 icu주말당직을 하지 않는 사람은 기본 2일을 하고, 추가로 필요하다면, 휴일 icu당직자에서 duty_score가 높은 사람 중에서 랜덤으로 선택.

>5) 휴일 call 당직은 icu당직을 하지 않는 멤버 그 다음은 duty_score가 낮은 사람을 배정

>> 5-1) 特定の人は週末Call_dutyをしないかも

>6) 평일 call 당직을 duty_score가 높은 사람부터 순차적으로 배정

>7) 모든 과정은 랜덤으로 배정하고, 금기사항을 지킬 수 없는 사항이면, 에러메시지를 표시하고, 처음부터 다시시작.

>8) Weight(6 classification)

>>: 연휴 첫날부터 연휴 마지막 날의 전날icu > 연휴 마지막날icu > 휴일 전날의 평일icu > 나머지 평일icu > 휴일 call당직 > 평일 call당직

>9) 주말에 icu나 call　당직은 한달에 2번만 들어가게 만들기

>10) 되도록이면, 주말의 icu 혹은 call당직으로 주말이 망가지는건 2번까지만 허용.

>
> 1)Deciding duty members on weekend and holydays

> 2)First, Decide icu_duty members

> 3)randomly distribution -> check unavailable_day -> check_within_2days after previous_duty

> 4)Second, Decide call_duty members

> 5)randomly distribution -> check unavailable_day -> check_within 2days after previous_duty

> 6)Third, Check if the duty pair can be able to handle cardiac surgery.

> 7)if they are unable to handle cardiac surgery, Restart alogorithm of deciding weekend duty members.

> 8)if there is 5 tries for deciding and we can decide members by algorithm, make a Error message to user.

## 5.1 Making a Dataframe based on collected information

In [257]:
members_df = make_members_info(df_mem_info, df_relation, df_trans_hoped_day)

#Check if this month requires old boy to do holiday duty twice. And manipulate holiday duty count of holiday duty of old boys

#이건 나중에 다시 생각해봐야 할듯 함!!!!

# members_df = select_twice_old_boy(df_mem_info = df_mem_info, 
#                                   info_days = info_days, 
#                                   members_df = members_df,
#                                   old_boy = old_boy)

#Apply infomation about no_holidas to members_info based on members_df
members_info = members_df.to_dict(orient="dict")

#Deepcopy 를 해야 origin_members_info의 데이터를 다른 메모리에 저장할 수 있음. dict의 경우에는 copy()도 얕은 복사임.
origin_members_info = copy.deepcopy(members_info)

OLD BOYS are ['北', '清水と', '荒井', '柴田', '井上', '上田']
No ICU members are ['北', '清水と', '荒井', '井上', '猪原']
No Cardiac members are ['北', '井上', '上田', '猪原', '小山']
Members of NO ADDITIONL duty are ['北', '井上', '上田', '猪原']


In [260]:
display(members_df)

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
old_boy,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
no_icu_duty,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
no_cardiac_surgery,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0
no_duty,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
no_additional_duty,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
weekend_icu,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1
weekend_call,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
num_icu_duty,0,0,0,3,0,0,1,3,3,0,3,3,3,3,3,3,3,3
num_call_duty,1,3,3,3,1,1,3,3,3,0,3,3,3,3,3,3,3,3
duty_num,1,3,3,3,1,4,4,4,4,4,4,4,4,4,4,4,4,4


In [270]:
members_df, icu_info_days, _ = hoped_duty(members_df, info_days, icu = True)
members_df, _, call_info_days = hoped_duty(members_df, info_days, icu = False)

day : 1, score = 4
上田's duty_score(after) : -34 -> -38
day : 15, score = 8
上田's duty_score(after) : -38 -> -46
day : 29, score = 4
上田's duty_score(after) : -46 -> -50
day : 24, score = 4
一澤's duty_score(after) : -34 -> -38
day : 2, score = 4
西村's duty_score(after) : -34 -> -38
day : 25, score = 4
孫's duty_score(after) : -34 -> -38
day : 3, score = 4
猪原's duty_score(after) : -34 -> -38
day : 13, score = 10
猪原's duty_score(after) : -38 -> -48
day : 21, score = 8
猪原's duty_score(after) : -48 -> -56
day : 10, score = 4
河合's duty_score(after) : -34 -> -38
day : 17, score = 4
竹田's duty_score(after) : -34 -> -38
day : 20, score = 6
北's duty_score(after) : -34 -> -40
day : 6, score = 6
井上's duty_score(after) : -40 -> -46
day : 27, score = 6
一澤's duty_score(after) : -46 -> -52
day : 17, score = 3
猪原's duty_score(after) : -52 -> -55


In [262]:
display(members_df)

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
old_boy,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
no_icu_duty,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
no_cardiac_surgery,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0
no_duty,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
no_additional_duty,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
weekend_icu,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1
weekend_call,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
num_icu_duty,0,0,0,3,0,0,1,3,3,0,3,3,3,3,3,3,3,3
num_call_duty,1,3,3,3,1,1,3,3,3,0,3,3,3,3,3,3,3,3
duty_num,1,3,3,3,1,4,4,4,4,4,4,4,4,4,4,4,4,4


## 5.2 Apply data of hoped day to members_info dataframe

In [3134]:
#희망일자부터 처리!
#만약에 겹치면, 일정표 짜기전에 서로 상의해서 한명만 남기도록 하기!

#if there is a with to do duty on specified days, first of all, distribute duty days members wished
icu_info_days = info_days.copy()
icu_info_days[0] = icu_info_days[0]
icu_info_days[1] = icu_info_days[1]
icu_info_days[2] = icu_info_days[2]
icu_info_days[3] = icu_info_days[3]

call_info_days = info_days.copy()
call_info_days[0] = call_info_days[0]
call_info_days[1] = call_info_days[1]
call_info_days[2] = call_info_days[2]
call_info_days[3] = call_info_days[3]

for name in members_info.keys():
    if members_info[name]['hope_icu_duty'] :
        for day in members_info[name]['hope_icu_duty']:
            members_info[name]['duty_score'] = count_down_duty_score(name,
                                                                            day,
                                                                            members_info,
                                                                            info_days,
                                                                            icu=True)
            if day not in info_days[3]:
                members_info[name]['no_holidays'] +=1
            members_info[name]['duty_num'] -=1


        members_info[name]['icu_duty'] = members_info[name]['hope_icu_duty']
        members_info[name]['hope_icu_duty'] = []
        icu_info_days[0] = remove_list(icu_info_days[0], members_info[name]['icu_duty'])
        icu_info_days[1] = remove_list(icu_info_days[1], members_info[name]['icu_duty'])
        icu_info_days[2] = remove_list(icu_info_days[2], members_info[name]['icu_duty'])
        icu_info_days[3] = remove_list(icu_info_days[3], members_info[name]['icu_duty'])
for name in members_info.keys():
    if members_info[name]['hope_call_duty'] :
        for day in members_info[name]['hope_call_duty']:
            members_info[name]['duty_score'] = count_down_duty_score(name,
                                                                            day,
                                                                            members_info,
                                                                            info_days,
                                                                            icu=False)
            if day in (info_days[0]+ info_days[1]):
                members_info[name]['no_holidays'] +=1
            members_info[name]['duty_num'] -=1
        members_info[name]['call_duty'] = members_info[name]['hope_call_duty']
        members_info[name]['hope_call_duty'] = []
        call_info_days[0] = remove_list(call_info_days[0], members_info[name]['call_duty'])
        call_info_days[1] = remove_list(call_info_days[1], members_info[name]['call_duty'])
        call_info_days[2] = remove_list(call_info_days[2], members_info[name]['call_duty'])
        call_info_days[3] = remove_list(call_info_days[3], members_info[name]['call_duty'])


day : 1, score = 4
上田's duty_score(after) : -120
day : 15, score = 8
上田's duty_score(after) : -128
day : 29, score = 4
上田's duty_score(after) : -132
day : 24, score = 4
一澤's duty_score(after) : -3
day : 2, score = 4
西村's duty_score(after) : 3
day : 25, score = 4
孫's duty_score(after) : 3
day : 3, score = 4
猪原's duty_score(after) : -129
day : 13, score = 10
猪原's duty_score(after) : -139
day : 21, score = 8
猪原's duty_score(after) : -147
day : 10, score = 4
河合's duty_score(after) : 3
day : 17, score = 4
竹田's duty_score(after) : 3
day : 20, score = 6
北's duty_score(after) : -112
day : 6, score = 6
井上's duty_score(after) : -112
day : 27, score = 6
一澤's duty_score(after) : -9
day : 17, score = 3
猪原's duty_score(after) : -150


In [263]:
display(members_df)

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
old_boy,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
no_icu_duty,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
no_cardiac_surgery,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0
no_duty,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
no_additional_duty,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
weekend_icu,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1
weekend_call,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
num_icu_duty,0,0,0,3,0,0,1,3,3,0,3,3,3,3,3,3,3,3
num_call_duty,1,3,3,3,1,1,3,3,3,0,3,3,3,3,3,3,3,3
duty_num,1,3,3,3,1,4,4,4,4,4,4,4,4,4,4,4,4,4


## 5.3 Assign ICU duty

In [3136]:
decide_duty_members(members_info, icu_info_days, interval = 3, icu= True)

[6, 14, 20, 27]に対してICU当直を決めます。
try number:  1
['小山', '池田', '寺西', '髙田', '李', '久保', '柴田']の中でNoneのICU当直を決めます
小山と27日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
27 小山
[27]
27 is unavailable day of 小山
小山は27日のicu当直ができません.
池田と6日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
6 池田
[]
Days is [3, 4, 5, 6, 7, 8, 9]
池田's ICU duty is []
[]
[]
remove 6 from [6, 14, 20, 27]
残りの当直日は[14, 20, 27]
day : 6, score = 10
池田's duty_score(after) : 1
池田は6日のICU当直ができます
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
寺西と27日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
27 寺西
[27]
27 is unavailable day of 寺西
寺西は27日のicu当直ができません.
髙田と20日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
20 髙田
[20]
20 is unavailable day of 髙田
髙田は20日のicu当直ができません.
李と20日のicu当直をチェックします.
 Max_score: 11, 実施回数：1
20 李
[]
Days is [17, 18, 19, 20, 21, 22, 23]
李's ICU duty is []
[]
[]
remove 20 from [14, 20, 27]
残りの当直日は[14, 27]
day : 20, score = 10
李's duty_score(after) : 1
李は20日のICU当直ができます
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [3137]:
df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[8, 12]","[6, 7, 12]","[5, 6, 7, 13, 20, 26, 27, 28]",[27],"[1, 15, 18, 19, 20, 21, 26, 27, 28, 29, 30, 31]","[11, 20, 21, 25]","[3, 7, 10, 12, 13, 14, 17, 18, 19, 20, 21, 28,...","[1, 4, 5, 6, 7, 13, 14, 29]","[3, 5, 6, 12, 13, 27, 28, 29]","[1, 2, 5, 6, 7, 9, 14, 15, 16, 19, 20, 23, 26,...","[5, 6, 7, 8, 12, 19, 20, 25, 26]","[1, 2, 8, 9, 11, 12, 13, 14, 15, 16, 21, 22, 2...","[6, 13, 14, 15, 19, 20, 21, 22]","[1, 2, 5, 14, 15, 26, 27, 28, 29]","[5, 6, 7, 12, 13, 20]","[1, 8, 15, 17, 18, 19, 20, 21, 22, 29]","[5, 6]","[1, 2]"
hope_icu_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
hope_call_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],"[28, 16, 8]",[],"[1, 15, 29]",[24],"[2, 26]","[25, 19]","[3, 13, 21]","[10, 23]","[5, 18]","[17, 12]","[7, 30]","[27, 4, 9]","[6, 11]","[14, 22]","[20, 31]"
call_duty,[20],[],[],[],[6],[],[27],[],[],[17],[],[],[],[],[],[],[],[]
duty_score,-112,5,5,-5,-112,-132,-9,-6,-6,-150,-1,-2,-6,-1,-7,-3,-3,-3


## 5.4 Assign Call duty

In [3138]:
decide_duty_members(members_info, call_info_days, interval = 2, icu = False)

[13, 14]に対してCALL当直を決めます。
try number:  1
['荒井', '清水と']の中でNoneのCall当直を決めます
荒井と13日のCall当直をチェックします.
 Max_score: 5, 実施回数：1
13 荒井
[13]
13 is unavailable day of 荒井
荒井は13日のcall当直ができません.
清水とと14日のCall当直をチェックします.
 Max_score: 5, 実施回数：1
14 清水と
[]
Days is [12, 13, 14, 15, 16]
清水と's CALL duty is []
[]
[]
久保 of icu_duty of 14  and  清水と of call_duty is able to handle Cardiac surgery.
Either 久保 or 清水と at 14 is not OLD BOY.
久保 of icu_duty of 14  and  清水と of call_duty have good relationship each other.
remove 14 from [13, 14]
残りの当直日は[13]
day : 14, score = 6
清水と's duty_score(after) : -1
清水とは14日のcall当直ができます
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
try number:  2
['荒井']の中で14のCall当直を決めます
荒井と13日のCall当直をチェックします.
 Max_score: 5, 実施回数：2
13 荒井
[13]
13 is unavailable day of 荒井
荒井は13日のcall当直ができません.
try number:  3
['荒井']の中で13のCall当直を決めます
荒井と13日のCall当直をチェックします.
 Max_score: 5, 実施回数：3
13 荒井
[13]
13 is unavailable day of 荒井
荒井は13日のcall当直ができません.
try number:  4
['荒井']の中で13のC

In [3139]:
df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[8, 12]","[6, 7, 12]","[5, 6, 7, 13, 20, 26, 27, 28]",[27],"[1, 15, 18, 19, 20, 21, 26, 27, 28, 29, 30, 31]","[11, 20, 21, 25]","[3, 7, 10, 12, 13, 14, 17, 18, 19, 20, 21, 28,...","[1, 4, 5, 6, 7, 13, 14, 29]","[3, 5, 6, 12, 13, 27, 28, 29]","[1, 2, 5, 6, 7, 9, 14, 15, 16, 19, 20, 23, 26,...","[5, 6, 7, 8, 12, 19, 20, 25, 26]","[1, 2, 8, 9, 11, 12, 13, 14, 15, 16, 21, 22, 2...","[6, 13, 14, 15, 19, 20, 21, 22]","[1, 2, 5, 14, 15, 26, 27, 28, 29]","[5, 6, 7, 12, 13, 20]","[1, 8, 15, 17, 18, 19, 20, 21, 22, 29]","[5, 6]","[1, 2]"
hope_icu_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
hope_call_duty,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],"[28, 16, 8]",[],"[1, 15, 29]",[24],"[2, 26]","[25, 19]","[3, 13, 21]","[10, 23]","[5, 18]","[17, 12]","[7, 30]","[27, 4, 9]","[6, 11]","[14, 22]","[20, 31]"
call_duty,[20],"[14, 30, 9]","[21, 10, 4]",[11],[6],[],"[27, 1]","[12, 16]","[7, 22]",[17],"[13, 29]","[31, 25]","[5, 24]","[19, 3]",[18],"[26, 2]","[28, 8]","[15, 23]"
duty_score,-112,-7,-7,-8,-112,-132,-12,-12,-15,-150,-10,-8,-12,-7,-10,-9,-12,-12


## 5.5 Make a Dataframe based on collected data and Assigning duty day by algorithm.

In [3140]:
for name in members_info.keys():
    if origin_members_info[f'{name}']['hope_icu_duty']:
        members_info[f'{name}']['hope_icu_duty'] = origin_members_info[f'{name}']['hope_icu_duty'] #+ members_info[f'{name}']['hope_icu_duty']
    if origin_members_info[f'{name}']['hope_call_duty']:
        members_info[f'{name}']['hope_call_duty'] = origin_members_info[f'{name}']['hope_call_duty'] #+ members_info[f'{name}']['hope_icu_duty']
    members_info[f'{name}']['icu_duty'] = sorted(members_info[f'{name}']['icu_duty'])
    members_info[f'{name}']['call_duty'] = sorted(members_info[f'{name}']['call_duty'])

df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[8, 12]","[6, 7, 12]","[5, 6, 7, 13, 20, 26, 27, 28]",[27],"[1, 15, 18, 19, 20, 21, 26, 27, 28, 29, 30, 31]","[11, 20, 21, 25]","[3, 7, 10, 12, 13, 14, 17, 18, 19, 20, 21, 28,...","[1, 4, 5, 6, 7, 13, 14, 29]","[3, 5, 6, 12, 13, 27, 28, 29]","[1, 2, 5, 6, 7, 9, 14, 15, 16, 19, 20, 23, 26,...","[5, 6, 7, 8, 12, 19, 20, 25, 26]","[1, 2, 8, 9, 11, 12, 13, 14, 15, 16, 21, 22, 2...","[6, 13, 14, 15, 19, 20, 21, 22]","[1, 2, 5, 14, 15, 26, 27, 28, 29]","[5, 6, 7, 12, 13, 20]","[1, 8, 15, 17, 18, 19, 20, 21, 22, 29]","[5, 6]","[1, 2]"
hope_icu_duty,[],[],[],[],[],"[1, 15, 29]",[24],[2],[25],"[3, 13, 21]",[10],[],[17],[],[],[],[],[]
hope_call_duty,[20],[],[],[],[6],[],[27],[],[],[17],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],"[8, 16, 28]",[],"[1, 15, 29]",[24],"[2, 26]","[19, 25]","[3, 13, 21]","[10, 23]","[5, 18]","[12, 17]","[7, 30]","[4, 9, 27]","[6, 11]","[14, 22]","[20, 31]"
call_duty,[20],"[9, 14, 30]","[4, 10, 21]",[11],[6],[],"[1, 27]","[12, 16]","[7, 22]",[17],"[13, 29]","[25, 31]","[5, 24]","[3, 19]",[18],"[2, 26]","[8, 28]","[15, 23]"
duty_score,-112,-7,-7,-8,-112,-132,-12,-12,-15,-150,-10,-8,-12,-7,-10,-9,-12,-12


# 6. Show tables

## 6.1 Table of List type of Duty days

In [3141]:
table = make_schedule_df(members_info)

   icu_duty call_duty
1        上田        一澤
2        西村        池田
3        猪原        寺西
4        髙田        荒井
5        小山        竹田
6        池田        井上
7        寺西         孫
8        柴田        久保
9        髙田       清水と
10       河合        荒井
11       池田        柴田
12       竹田        西村
13       猪原        河合
14       久保       清水と
15       上田         李
16       柴田        西村
17       竹田        猪原
18       小山        髙田
19        孫        寺西
20        李         北
21       猪原        荒井
22       久保         孫
23       河合         李
24       一澤        竹田
25        孫        小山
26       西村        池田
27       髙田        一澤
28       柴田        久保
29       上田        河合
30       寺西       清水と
31        李        小山


/var/folders/h5/jyq7r4dd38l9x866mmggyx280000gn/T/ipykernel_1010/1214954852.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  confirmed_schedule['icu_duty'][n] = name
/var/folders/h5/jyq7r4dd38l9x866mmggyx280000gn/T/ipykernel_1010/121495485

## 6.2 Table of Calendar type of Duty days


In [3142]:
calendar = make_calendar(table, year, month, start, end)


                    Mon  Tue Wed  Thu  Fri  Sat  Sun
Week     Duty                                       
1st week day          1    2   3    4    5   *6   *7
         icu_duty    上田   西村  猪原   髙田   小山   池田   寺西
         call_duty   一澤   池田  寺西   荒井   竹田   井上    孫
2st week day          8    9  10   11   12  *13  *14
         icu_duty    柴田   髙田  河合   池田   竹田   猪原   久保
         call_duty   久保  清水と  荒井   柴田   西村   河合  清水と
3st week day        *15   16  17   18   19  *20  *21
         icu_duty    上田   柴田  竹田   小山    孫    李   猪原
         call_duty    李   西村  猪原   髙田   寺西    北   荒井
4st week day         22   23  24   25   26  *27  *28
         icu_duty    久保   河合  一澤    孫   西村   髙田   柴田
         call_duty    孫    李  竹田   小山   池田   一澤   久保
5st week day         29   30  31  NaN  NaN  NaN  NaN
         icu_duty    上田   寺西   李  NaN  NaN  NaN  NaN
         call_duty   河合  清水と  小山  NaN  NaN  NaN  NaN


## 6.3 Make a members info dataframe Adding Marked duty days of no_holidays

In [3143]:
for name in members_info.values():
    # for n in name['icu_duty']:
    #     if n not in info_days[3]:
    #         name['no_holidays'] +=1
    # for n in name['call_duty']:
    #     if n in (info_days[0]+ info_days[1]):
    #         name['no_holidays'] +=1
    name['hope_icu_duty'] = [f'*{n}'  if n not in info_days[3] else n for n in name['hope_icu_duty']]
    name['hope_call_duty'] = [f'*{n}' if n not in info_days[3] else n for n in name['hope_call_duty']]
    name['icu_duty'] = [f'*{n}'  if n not in info_days[3] else n for n in name['icu_duty']]
    name['call_duty'] = [f'*{n}' if n not in info_days[3] else n for n in name['call_duty']]

In [3144]:
df = pd.DataFrame(members_info)
df

,北,清水と,荒井,柴田,井上,上田,一澤,西村,孫,猪原,河合,小山,竹田,寺西,髙田,池田,久保,李
no_icu_duty,True,True,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False
no_cardiac_surgery,True,False,False,False,True,True,False,False,False,True,False,True,False,False,False,False,False,False
no_additional_duty,True,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
old_boy,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
unavailable_days,"[8, 12]","[6, 7, 12]","[5, 6, 7, 13, 20, 26, 27, 28]",[27],"[1, 15, 18, 19, 20, 21, 26, 27, 28, 29, 30, 31]","[11, 20, 21, 25]","[3, 7, 10, 12, 13, 14, 17, 18, 19, 20, 21, 28,...","[1, 4, 5, 6, 7, 13, 14, 29]","[3, 5, 6, 12, 13, 27, 28, 29]","[1, 2, 5, 6, 7, 9, 14, 15, 16, 19, 20, 23, 26,...","[5, 6, 7, 8, 12, 19, 20, 25, 26]","[1, 2, 8, 9, 11, 12, 13, 14, 15, 16, 21, 22, 2...","[6, 13, 14, 15, 19, 20, 21, 22]","[1, 2, 5, 14, 15, 26, 27, 28, 29]","[5, 6, 7, 12, 13, 20]","[1, 8, 15, 17, 18, 19, 20, 21, 22, 29]","[5, 6]","[1, 2]"
hope_icu_duty,[],[],[],[],[],"[1, *15, 29]",[24],[2],[25],"[3, *13, *21]",[10],[],[17],[],[],[],[],[]
hope_call_duty,[*20],[],[],[],[*6],[],[*27],[],[],[17],[],[],[],[],[],[],[],[]
icu_duty,[],[],[],"[8, 16, *28]",[],"[1, *15, 29]",[24],"[2, *26]","[*19, 25]","[3, *13, *21]","[10, 23]","[*5, 18]","[*12, 17]","[*7, 30]","[4, 9, *27]","[*6, 11]","[*14, 22]","[*20, 31]"
call_duty,[*20],"[9, *14, 30]","[4, 10, *21]",[11],[*6],[],"[1, *27]","[*12, 16]","[*7, 22]",[17],"[*13, 29]","[25, 31]","[*5, 24]","[3, *19]",[18],"[2, *26]","[8, *28]","[*15, 23]"
duty_score,-112,-7,-7,-8,-112,-132,-12,-12,-15,-150,-10,-8,-12,-7,-10,-9,-12,-12


In [3145]:
calendar

Mon  Tue Wed  Thu  Fri  Sat  Sun
Week     Duty                                       
1st week day          1    2   3    4    5   *6   *7
         icu_duty    上田   西村  猪原   髙田   小山   池田   寺西
         call_duty   一澤   池田  寺西   荒井   竹田   井上    孫
2st week day          8    9  10   11   12  *13  *14
         icu_duty    柴田   髙田  河合   池田   竹田   猪原   久保
         call_duty   久保  清水と  荒井   柴田   西村   河合  清水と
3st week day        *15   16  17   18   19  *20  *21
         icu_duty    上田   柴田  竹田   小山    孫    李   猪原
         call_duty    李   西村  猪原   髙田   寺西    北   荒井
4st week day         22   23  24   25   26  *27  *28
         icu_duty    久保   河合  一澤    孫   西村   髙田   柴田
         call_duty    孫    李  竹田   小山   池田   一澤   久保
5st week day         29   30  31  NaN  NaN  NaN  NaN
         icu_duty    上田   寺西   李  NaN  NaN  NaN  NaN
         call_duty   河合  清水と  小山  NaN  NaN  NaN  NaN

## 6.4 Make a excel file of above tables

In [3146]:
with pd.ExcelWriter(f'//Users/taehunlee/Desktop/duty_schedule_ver2/output{year}_{month}.xlsx') as writer:
    calendar.to_excel(writer, sheet_name='Sheet1', index=True)
    table.to_excel(writer, sheet_name='Sheet2', index=True)
    df.to_excel(writer, sheet_name='Sheet3', index=True)